In [ ]:
#import libraries
import pandas as pd
import numpy as np
pd.set_option('display.float_format', lambda x: '%.4f' % x)
import matplotlib.pyplot as plt
import math
import os
import sys

# Preprocessing 

Code to preprocess the data of the selected experiments. To set desired experiments, filters, sensors and options, use next lines of code:

In [ ]:
#Set option

#Set sensors: set sensors to consider by setting their variable to 'True'. For ITSC please use the configuration: range_sens='False', cam_w_yolo_FT='True', cam_d_yolo_FT='True', cam_w_yolo_v5='True', cam_d_yolo_v5='True'
range_sens='False' 
cam_w_yolo_FT='True'
cam_d_yolo_FT='True'
cam_w_yolo_v5='True'
cam_d_yolo_v5='True'

#Set experiments between 0 and 11. Multiple experiments can be set:
exp_list=[0,1,2,3,4,5,6,7,8,9,10,11]

#set filters: between 'original','bright_1','bright_2','bright_3','dark_1','dark_2','dark_3','fog_1','fog_2','fog_3','rain_1','rain_2','rain_3'. Multiple filters can be set:
filter_list=['original','bright_1','bright_2','bright_3','dark_1','dark_2','dark_3','fog_1','fog_2','fog_3','rain_1','rain_2','rain_3']

#set folder name where the dataset is located
folder= "safe_road_data"

print("exp_list="+ str(exp_list))
print("filter_list="+ str(filter_list))
print("folder="+ str(folder))


In [ ]:

## Danger Function definition
def d_f(distance):
    if distance <= 0:
        return 0
    return 1 / math.log(distance + 0.6)

def d_s(speed):
    s_min = 0.05
    s_max = 0.5
    if speed <= s_min:
        return 0
    if speed >= s_max:
        return 1
    return (speed-s_min)/(s_max - s_min)

def d_a(acceleration):
    a_min = 1
    a_max = 10
    if acceleration <= - a_max:
        return -1
    elif -a_max < acceleration <= -a_min:
        return (acceleration+a_min)/(a_max - a_min)
    elif -a_min < acceleration <= +a_min:
        return 0
    elif a_min < acceleration <= +a_max:
        return (acceleration - a_min)/(a_max - a_min)
    elif acceleration > a_max:
        return 1
    return 0
    #if acceleration < 1 and acceleration > -1: # -1 < a < 1
        #return 1
    #elif acceleration < -1:
        #return 0.9
    #else:
        #return 1.1

def danger_function_distance(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return distance_factor

def danger_function_speed(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return speed_factor

def danger_function_acceleration(row, sensor ,k1 = 1, k2 = 1, k3 = 1):
    distance = row[f'distance']+ 0.000001
    speed = row[f'speed']
    acceleration = row[f'acceleration']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return acceleration_factor

def danger_function_composed(row,sensor ,k1 = 0.1):
    distance = row[f'distance_{sensor}'] + 0.000001
    speed = row[f'speed_{sensor}']
    acceleration = row[f'acceleration_{sensor}']
    distance_factor = d_f(distance)
    speed_factor = d_s(speed)
    acceleration_factor = d_a(acceleration)
    return (speed_factor + k1 * acceleration_factor) * distance_factor


In [ ]:
last_frame_w=[190, 263, 206, 205, 167, 260, 367, 237, 188, 269, 251, 233, 304, 454, 190]
last_frame_d=[190, 262, 203, 208, 166, 260, 367, 237, 189, 273, 254, 235, 307, 455, 190]


In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)



In [ ]:
pd.reset_option('all')

In [ ]:
interval_end_array=np.zeros(15)
for exp_nr in exp_list:
    folder_name=f"{folder}/exp_{exp_nr}"
    df_tracker = pd.read_csv(f'{folder_name}/tracker{exp_nr}.csv')

    #time is converted from timestamps to seconds
    timestamp_track_min = df_tracker['timestamp'].copy(deep=True).min()


    if (cam_w_yolo_FT== 'True') or (cam_w_yolo_v5=='True') or (cam_d_yolo_FT== 'True') or (cam_d_yolo_v5=='True'):
        # timestamps of camera frame coming from wheelchair and drone
        df_wheelchair_drone_stamps = pd.read_csv(f'{folder_name}/camera_stamps{exp_nr}.csv')
        

    if (cam_w_yolo_FT== 'True') or (cam_w_yolo_v5=='True'):
        #Correcting incongruencies in data dimensions between timestamps and frame indices 
        check_nan_stamp_w = df_wheelchair_drone_stamps['stamp_w'].isnull().sum()
        if check_nan_stamp_w!=0:
            df_wheelchair_stamps = df_wheelchair_drone_stamps['stamp_w'].head(-check_nan_stamp_w)
        else:
            df_wheelchair_stamps =df_wheelchair_drone_stamps['stamp_w']
        #df_wheelchair = df_wheelchair_temp
        
    if (cam_d_yolo_FT== 'True') or (cam_d_yolo_v5=='True'):
        check_nan_stamp_d = df_wheelchair_drone_stamps['stamp_d'].isnull().sum()
        if check_nan_stamp_d!=0:
            df_drone_stamps = df_wheelchair_drone_stamps['stamp_d'].head(-check_nan_stamp_d)
        else:
            df_drone_stamps =df_wheelchair_drone_stamps['stamp_d']
        
        #Reset Dataframe index, correcting data type
        df_drone_stamps = df_drone_stamps.astype(float)

    df_drone_FT =pd.DataFrame()
    df_drone_FT['timestamp'] =df_drone_stamps
    #time is converted from timestamps to seconds
    df_drone_FT.index = (df_drone_FT['timestamp'] - timestamp_track_min)/1_000_000_000

    df_drone_FT.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True)

    df_wheelchair_v5 =pd.DataFrame()
    df_wheelchair_v5['timestamp'] =df_wheelchair_stamps
    df_wheelchair_v5.index = (df_wheelchair_v5['timestamp'] - timestamp_track_min)/1_000_000_000

    df_wheelchair_v5.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True)


    interval_end_array[exp_nr]= min(df_drone_FT.index[last_frame_d[exp_nr]],df_wheelchair_v5.index[last_frame_w[exp_nr]])
    print(f'exp {exp_nr} last_drone=',df_drone_FT.index[last_frame_d[exp_nr]],'  last_wheel=', df_wheelchair_v5.index[last_frame_w[exp_nr]],'last_min=', min(df_drone_FT.index[last_frame_d[exp_nr]],df_wheelchair_v5.index[last_frame_w[exp_nr]]))


In [ ]:
#interval_end_array=np.zeros(15)
interval_start_array=np.zeros(15)
focal_v8=1100
focal_v5=1200
for exp_nr in exp_list:

    for filter in filter_list:
        #set folder name

        folder_name= f"{folder}/exp_{exp_nr}"
        

        #parameter of interest for all sensors:
        max_speed = 3.5
        min_speed = 0 # We assume the obstacle is not going backwards
        # _______________________________________________________________________________________________
        # Data from TRACKER:
        # timestamp | distance
        df_tracker = pd.read_csv(f'{folder_name}/tracker{exp_nr}.csv')

        #time is converted from timestamps to seconds
        timestamp_track_min = df_tracker['timestamp'].copy(deep=True).min()
        #print(timestamp_track_min)
        df_tracker.index = (df_tracker['timestamp'] - timestamp_track_min)/1_000_000_000
        df_tracker.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True) 

        #Speed and accelaration calculus for the tracker
        df_tracker['speed_tracker'] = np.insert(- np.diff(df_tracker['distance'])/np.diff(df_tracker.index), 0, 0)
        df_tracker['acceleration_tracker'] = np.insert(np.diff(df_tracker['speed_tracker'])/np.diff(df_tracker.index), 0, 0)

        ## Truncated experiment in the range of interest
        #To limit the analisys of the experiment to relevant intervals where the obstacle is approaching.
        #Set the interval of interest With respect to number experiment:
        #print( f'exp{exp_nr} distance min='+str(df_tracker['distance'].min()) + '; index_time='+str(df_tracker['distance'].idxmin()))
        #print( f'exp{exp_nr} distance min='+str(df_tracker['distance'].idxmin()))


        


    

        ################df_tracker = df_tracker.loc[:interval_end]
        
        df_tracker = df_tracker.drop(['absolute_timestamp'], axis=1)
        df_tracker.rename(columns={'distance': 'distance_tracker'}, inplace=True)
        

        max_value_tracker=df_tracker['distance_tracker'].max()

        #first choice for inrval end:   interval_end = df_tracker['distance'].idxmin()

        interval_start = df_tracker[df_tracker['speed_tracker']>0.05].any(axis=1).index[0]-0.3
        interval_start_array[exp_nr]=interval_start
        interval_end = df_tracker.index[df_tracker['distance_tracker'] == df_tracker['distance_tracker'].min()].tolist()[0]
        if exp_nr==5:
            interval_end=5.5
        interval_end_array[exp_nr]=interval_end
        #interval_end=None

        # fig, ax = plt.subplots(3,1)
        # plt.title('Tracker')
        # # Index 4 Axes arrays in 4 subplots within 1 Figure: 
        # ax[0].plot(df_tracker.index, df_tracker['distance'], label='tracker distance') 
        # ax[0].set_ylabel('Distance [m]')
        # ax[0].set_xlabel('Time [s]')
        # #ax[0].set_xlim(2,8)
        # ax[0].grid()

        # ax[1].plot(df_tracker.index, df_tracker['speed_tracker'], label='speed') 
        # ax[1].set_ylabel('Speed [m/s]')
        # #ax[1].set_ylim(-0.1,0.1)
        # #ax[1].set_xlim(2,8)
        # ax[1].set_xlabel('Time [s]')
        # ax[1].grid()

        # ax[2].plot(df_tracker.index, df_tracker['acceleration_tracker'], label='acceleration')
        # ax[2].set_ylabel('Acceleration [m/s${^2}$]')
        # ax[2].set_xlabel('Time [s]')
        # ax[2].grid()
        df_tracker = df_tracker.loc[interval_start:interval_end]

         
        # _______________________________________________________________________________________________
        if range_sens=='True':
            # Data from RANGE SENSORS :
            #timestamp | range_0 | range_1 | range_2 | range_3
            
            df_range = pd.read_csv(f'{folder_name}/range_{exp_nr}.csv')
            #df_range = pd.read_csv('safe_road_data/exp_14/range_14.csv')
            
            #Range sensors fusion
            df_range['distance'] = df_range.min(axis=1)
            df_range = df_range.drop(['range_0', 'range_1', 'range_2', 'range_3'], axis=1)
            
            #time is converted from timestamps to seconds for range sensors
            #print(timestamp_track_min)
            df_range.index = (df_range['timestamp'] - timestamp_track_min)/1_000_000_000
            #df_range.index = (df_range['timestamp'] - df_tracker['timestamp'].min())/1_000_000_000

            df_range.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True)
            


            # Automatic detection end of corrupted sample for range sensors unit
            
            range_error_index=[]
            range_error=[]
            range_error_time=[]

            for i in np.arange(len(df_range['distance'])-2):
                if df_range['distance'].iloc[i+2]- df_range['distance'].iloc[i]>0.25:
                    #print('PICCO RILEVATO')
                    #print('i='+str(i))
                    range_error_index_prova=[]
                    j=0
                    while (j< 5):
                        if i+1+j>len(df_range['distance'])-2:
                            #print('FINE FORZATA')
                            break
                        include='no'
                        range_error_index_prova.append(i+1+j)
                        #print('i+1+j='+str(i+1+j))

                        if df_range['distance'].iloc[i+1+j]- df_range['distance'].iloc[i+1+j+1]>0.2:
                            include='yes'
                            break  
                        j+=1

                    if include=='yes':
                        range_error_index.append(range_error_index_prova)

            
            j=0
            x_limits=[]
            for i in range_error_index:
                x_limits.append(list([df_range.index[i[0]] - 0.5,df_range.index[i[-1]] + 0.5]))
                j+=1

            for set_error in range_error_index:
                df_range=df_range.drop(df_range.index[set_error],axis=0)

            ## Truncated experiment in the range of interest
            #To limit the analisys of the experiment to relevant intervals where the obstacle is approaching.
            #Set the interval of interest With respect to number experiment:
            

            df_range = df_range.loc[:interval_end]



            ## Smoothing
            #Apply rolling average with the previous n values, 5 for the range sensor, and 2 for the cameras, in order to reduce outliers

            df_range['distance'] = df_range['distance'].rolling(window=5).mean()
            df_range = df_range.rename(columns={'distance': 'distance_range','absolute_timestamp': 'absolute_timestamp_range'})

            #Speed and accelaration calculus for RANGE
            max_speed = 3.5
            min_speed = 0 # We assume the obstacle is not going backwards

            df_range['speed_range'] = np.insert(- np.diff(df_range['distance_range'])/np.diff(df_range.index), 0, 0)
            df_range['speed_range'] = np.clip(df_range['speed_range'], a_min=min_speed, a_max=max_speed)
            df_range['acceleration_range'] = np.insert(np.diff(df_range['speed_range'])/np.diff(df_range.index), 0, 0)

            df_range['speed_range'] = np.insert(- np.diff(df_range['distance_range'])/np.diff(df_range.index), 0, 0)
            df_range['acceleration_range'] = np.insert(np.diff(df_range['speed_range'])/np.diff(df_range.index), 0, 0)

            #danger function application
            df_range['danger_range'] = df_range.apply(lambda row: danger_function_composed(row, 'range'), axis=1)


            df_range = df_range.drop(['absolute_timestamp_range'], axis=1)
            #print('df_range at'+ str(exp_nr)+'='+str(df_range))
            #print('OK FINO A ORA')
            
  


            # fig, ax = plt.subplots(3,1)
            # # Index 4 Axes arrays in 4 subplots within 1 Figure: 
            # plt.title('Range')
            # ax[0].plot(df_range.index, df_range['distance_range'], label='range distance') 
            # ax[0].set_ylabel('Distance [m]')
            # ax[0].set_xlabel('Time [s]')
            # ax[0].grid()

            # ax[1].plot(df_range.index, df_range['speed_range'], label='range speed') 
            # ax[1].set_ylabel('Speed [m/s]')
            # ax[1].set_xlabel('Time [s]')
            # ax[1].grid()
            # ax[2].plot(df_range.index, df_range['acceleration_range'], label='range acceleration')
            # ax[2].set_ylabel('Acceleration [m/s${^2}$]')
            # ax[2].set_xlabel('Time [s]')
            # ax[2].grid()


            # plt.show()  

        # _______________________________________________________________________________________________ 
        if (cam_w_yolo_FT== 'True') or (cam_w_yolo_v5=='True') or (cam_d_yolo_FT== 'True') or (cam_d_yolo_v5=='True'):
            # timestamps of camera frame coming from wheelchair and drone
            df_wheelchair_drone_stamps = pd.read_csv(f'{folder_name}/camera_stamps{exp_nr}.csv')
            

        if (cam_w_yolo_FT== 'True') or (cam_w_yolo_v5=='True'):
            #Correcting incongruencies in data dimensions between timestamps and frame indices 
            check_nan_stamp_w = df_wheelchair_drone_stamps['stamp_w'].isnull().sum()
            if check_nan_stamp_w!=0:
                df_wheelchair_stamps = df_wheelchair_drone_stamps['stamp_w'].head(-check_nan_stamp_w)
            else:
                df_wheelchair_stamps =df_wheelchair_drone_stamps['stamp_w']
            #df_wheelchair = df_wheelchair_temp
            
        if (cam_d_yolo_FT== 'True') or (cam_d_yolo_v5=='True'):
            check_nan_stamp_d = df_wheelchair_drone_stamps['stamp_d'].isnull().sum()
            if check_nan_stamp_d!=0:
                df_drone_stamps = df_wheelchair_drone_stamps['stamp_d'].head(-check_nan_stamp_d)
            else:
                df_drone_stamps =df_wheelchair_drone_stamps['stamp_d']
            
            #Reset Dataframe index, correcting data type
            df_drone_stamps = df_drone_stamps.astype(float)
        
        # _______________________________________________________________________________________________
        #bounding boxes generated by YOLO from drone videos in normal condition and after application of the several filters
        # index|original|bright_1|bright_2|bright_3|dark_1|dark_2|dark_3|fog_1|fog_2|fog_3|rain_1|rain_2|rain_3
        #YOLO FT
        if (cam_d_yolo_FT== 'True') :

            df_drone_temp_FT = pd.read_csv(f'{folder_name}/bbox_drone_FT.csv') #da cambiare nome su dataset
           
            
            if len(df_drone_temp_FT.index)!= len(df_drone_stamps.index):
                print("ERROR IN DATA DIMENSION FOR DRONE VIDEO")
                print("exp_nr="+ str(exp_nr))
                print("filter="+ str(filter))
                #sys.exit("Error message")

            #Reset Dataframe index, correcting data type
            df_drone_temp_FT = df_drone_temp_FT.reset_index(drop=True)
            

            df_drone_FT =pd.DataFrame()
            df_drone_FT['timestamp'] =df_drone_stamps

            #Filter selection between:original,bright_1,bright_2,bright_3,dark_1,dark_2,dark_3,fog_1,fog_2,fog_3,rain_1,rain_2,rain_3
            df_drone_FT['bbox'] =df_drone_temp_FT[filter]
            #distance computation
            df_drone_FT['distance'] = ((0.24 * focal_v8) / df_drone_FT['bbox'])                #time is converted from timestamps to seconds
            print('___________________________')
            print(df_drone_FT)
            #time is converted from timestamps to seconds
            df_drone_FT.index = (df_drone_FT['timestamp'] - timestamp_track_min)/1_000_000_000

            df_drone_FT.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True)
            
            #Speed and accelaration calculus for the tracker
            df_drone_FT = df_drone_FT.loc[interval_start:interval_end]
            
            
            
            ###############df_drone_FT = df_drone_FT.loc[:interval_end]
 
            
            # Delete the corrupted data from drone camera outputs:
            df_drone_clean = pd.DataFrame(columns=df_drone_FT.columns).copy(deep=True)
            bad_index = []
            for i in np.arange(len(df_drone_FT.index)):
                if abs(df_drone_FT.index[i] - df_drone_FT.index[i-1]) >= 0.01:
                    df_drone_clean = pd.concat([df_drone_clean, df_drone_FT.iloc[i-1].to_frame().T])

                else:
                    index_mean = (df_drone_FT.index[i] + df_drone_FT.index[i - 1]) / 2
                    temp_dict ={}
                    temp_dict['index'] = index_mean
                    temp_dict['distance'] = (df_drone_FT['distance'].iloc[i] + df_drone_FT['distance'].iloc[i-1])/2
                    temp_dict['bbox'] = (df_drone_FT['bbox'].iloc[i] + df_drone_FT['bbox'].iloc[i-1])/2
                    temp_dict['absolute_timestamp'] = (df_drone_FT['absolute_timestamp'].iloc[i] + df_drone_FT['absolute_timestamp'].iloc[i-1])/2
                    bad_index.append(df_drone_FT.index[i-1])
                    temp = pd.DataFrame(temp_dict, index=[temp_dict['index']]).copy(deep=True)
                    df_drone_clean = pd.concat([df_drone_clean, temp.drop(['index'], axis=1)])

            df_drone_FT = df_drone_FT.drop(bad_index)
            ## Smoothing
            #Apply rolling average with the previous n values, 5 for the range sensor, and 2 for the cameras, in order to reduce outliers
            df_drone_FT['distance'] = df_drone_FT['distance'].rolling(window=2).mean()
            df_drone_FT = df_drone_FT.rename(columns={'distance': 'distance_drone','absolute_timestamp': 'absolute_timestamp_drone', 'bbox': 'bbox_drone'})


 
            #Speed and accelaration calculus
            df_drone_FT['speed_drone'] = np.insert(- np.diff(df_drone_FT['distance_drone'])/np.diff(df_drone_FT.index), 0, 0)
            max_speed = 3.5
            min_speed = 0
            df_drone_FT['speed_drone'] = np.clip(df_drone_FT['speed_drone'], a_min=min_speed, a_max=max_speed)
            df_drone_FT['acceleration_drone'] = np.insert(np.diff(df_drone_FT['speed_drone'])/np.diff(df_drone_FT.index), 0, 0)

            #danger function application
            df_drone_FT['danger_drone'] = df_drone_FT.apply(lambda row: danger_function_composed(row, 'drone'), axis=1)
            
            df_drone_FT.rename(columns={'distance_drone': 'distance_drone_FT'}, inplace=True)
            df_drone_FT.rename(columns={'speed_drone': 'speed_drone_FT'}, inplace=True)
            df_drone_FT.rename(columns={'acceleration_drone': 'acceleration_drone_FT'}, inplace=True)
            df_drone_FT.rename(columns={'danger_drone': 'danger_drone_FT'}, inplace=True)
            
            df_drone_FT = df_drone_FT.drop(['bbox_drone','absolute_timestamp_drone'], axis=1)

            # fig, ax = plt.subplots(3,1)
            # # Index 4 Axes arrays in 4 subplots within 1 Figure: 
            # plt.title('Drone')
            # ax[0].plot(df_drone_FT.index, df_drone_FT['distance_drone'], label='drone distance') 
            # ax[0].plot(df_tracker.index, df_tracker['distance'], label='tracker distance') 
            # ax[0].set_ylabel('Distance [m]')
            # ax[0].set_xlabel('Time [s]')
            # ax[0].grid()
            # ax[0].set_xlim([0,9])

            # ax[1].plot(df_drone_FT.index, df_drone_FT['speed_drone'], label='drone speed')
            # ax[1].plot(df_tracker.index, df_tracker['speed_tracker'], label='speed') 
            # ax[1].set_ylabel('Speed [m/s]')
            # ax[1].set_xlabel('Time [s]')
            # ax[1].grid()
            # ax[1].set_xlim([0,9])

            # ax[2].plot(df_drone_FT.index, df_drone_FT['acceleration_drone'], label='drone acceleration')
            # ax[2].plot(df_tracker.index, df_tracker['acceleration_tracker'], label='acceleration')
            # ax[2].set_ylabel('Acceleration [m/s${^2}$]')
            # ax[2].set_xlabel('Time [s]')
            # ax[2].grid()
            # ax[2].set_xlim([0,9])
            # plt.show()
            
        # _______________________________________________________________________________________________
        #bounding boxes generated by YOLO from drone videos in normal condition and av5er application of the several filters
        # index|original|bright_1|bright_2|bright_3|dark_1|dark_2|dark_3|fog_1|fog_2|fog_3|rain_1|rain_2|rain_3
        #YOLO v5
        if cam_d_yolo_v5=='True':
            df_drone_temp_v5 = pd.read_csv(f'{folder_name}/bbox_drone_v5.csv') #da cambiare nome su dataset
            
            
            if len(df_drone_temp_v5.index)!= len(df_drone_stamps.index):
                print("ERROR IN DATA DIMENSION FOR DRONE VIDEO")
                print("exp_nr="+ str(exp_nr))
                print("filter="+ str(filter))
                #sys.exit("Error message")

            #Reset Dataframe index, correcting data type
            df_drone_temp_v5 = df_drone_temp_v5.reset_index(drop=True)
            

            df_drone_v5 =pd.DataFrame()
            df_drone_v5['timestamp'] =df_drone_stamps

            #Filter selection between:original,bright_1,bright_2,bright_3,dark_1,dark_2,dark_3,fog_1,fog_2,fog_3,rain_1,rain_2,rain_3
            df_drone_v5['bbox'] =df_drone_temp_v5[filter]
            #distance computation
            df_drone_v5['distance'] = ((0.24 * focal_v5) / df_drone_v5['bbox'])                #time is converted from timestamps to seconds
            
            #time is converted from timestamps to seconds
            df_drone_v5.index = (df_drone_v5['timestamp'] - timestamp_track_min)/1_000_000_000

            df_drone_v5.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True)

            #Speed and accelaration calculus for the tracker
            df_drone_v5 = df_drone_v5.loc[interval_start:interval_end]
            
            #################df_drone_v5 = df_drone_v5.loc[:interval_end]
            
            
            # Delete the corrupted data from drone camera outputs:
            df_drone_clean = pd.DataFrame(columns=df_drone_v5.columns).copy(deep=True)
            bad_index = []
            for i in np.arange(len(df_drone_v5.index)):
                if abs(df_drone_v5.index[i] - df_drone_v5.index[i-1]) >= 0.01:
                    df_drone_clean = pd.concat([df_drone_clean, df_drone_v5.iloc[i-1].to_frame().T])

                else:
                    index_mean = (df_drone_v5.index[i] + df_drone_v5.index[i - 1]) / 2
                    temp_dict ={}
                    temp_dict['index'] = index_mean
                    temp_dict['distance'] = (df_drone_v5['distance'].iloc[i] + df_drone_v5['distance'].iloc[i-1])/2
                    temp_dict['bbox'] = (df_drone_v5['bbox'].iloc[i] + df_drone_v5['bbox'].iloc[i-1])/2
                    temp_dict['absolute_timestamp'] = (df_drone_v5['absolute_timestamp'].iloc[i] + df_drone_v5['absolute_timestamp'].iloc[i-1])/2
                    bad_index.append(df_drone_v5.index[i-1])
                    temp = pd.DataFrame(temp_dict, index=[temp_dict['index']]).copy(deep=True)
                    df_drone_clean = pd.concat([df_drone_clean, temp.drop(['index'], axis=1)])

            df_drone_v5 = df_drone_v5.drop(bad_index)
            ## Smoothing
            #Apply rolling average with the previous n values, 5 for the range sensor, and 2 for the cameras, in order to reduce outliers
            df_drone_v5['distance'] = df_drone_v5['distance'].rolling(window=2).mean()
            df_drone_v5 = df_drone_v5.rename(columns={'distance': 'distance_drone','absolute_timestamp': 'absolute_timestamp_drone', 'bbox': 'bbox_drone'})



            #Speed and accelaration calculus
            df_drone_v5['speed_drone'] = np.insert(- np.diff(df_drone_v5['distance_drone'])/np.diff(df_drone_v5.index), 0, 0)
            #df_drone_v5['speed_drone'] = (- np.diff(df_drone_v5['distance_drone'])/np.diff(df_drone_v5.index)
            #print('np.diff(df_drone_v5.index)=', np.diff(df_drone_v5.index),'np.diff(df_drone_v5[distance_drone])=',np.diff(df_drone_v5['distance_drone']),'df_drone_v5[speed_drone]=',- np.diff(df_drone_v5['distance_drone'])/np.diff(df_drone_v5.index))
            max_speed = 3.5
            min_speed = 0
            df_drone_v5['speed_drone'] = np.clip(df_drone_v5['speed_drone'], a_min=min_speed, a_max=max_speed)
            df_drone_v5['acceleration_drone'] = np.insert(np.diff(df_drone_v5['speed_drone'])/np.diff(df_drone_v5.index), 0, 0)



            #danger function application
            df_drone_v5['danger_drone'] = df_drone_v5.apply(lambda row: danger_function_composed(row, 'drone'), axis=1)


            df_drone_v5.rename(columns={'distance_drone': 'distance_drone_v5'}, inplace=True)
            df_drone_v5.rename(columns={'speed_drone': 'speed_drone_v5'}, inplace=True)
            df_drone_v5.rename(columns={'acceleration_drone': 'acceleration_drone_v5'}, inplace=True)
            df_drone_v5.rename(columns={'danger_drone': 'danger_drone_v5'}, inplace=True)
            df_drone_v5 = df_drone_v5.drop(['bbox_drone','absolute_timestamp_drone'], axis=1)

            # fig, ax = plt.subplots(3,1)
            # # Index 4 Axes arrays in 4 subplots within 1 Figure: 
            # plt.title('Drone')
            # ax[0].plot(df_drone_v5.index, df_drone_v5['distance_drone_v5'], label='drone distance') 
            # ax[0].plot(df_tracker.index, df_tracker['distance_tracker'], label='tracker distance') 
            # ax[0].set_ylabel('Distance [m]')
            # ax[0].set_xlabel('Time [s]')
            # ax[0].grid()
            # ax[0].set_xlim([0,9])

            # ax[1].plot(df_drone_v5.index, df_drone_v5['speed_drone_v5'], label='drone speed')
            # ax[1].plot(df_tracker.index, df_tracker['speed_tracker'], label='speed') 
            # ax[1].set_ylabel('Speed [m/s]')
            # ax[1].set_xlabel('Time [s]')
            # ax[1].grid()
            # ax[1].set_xlim([0,9])

            # ax[2].plot(df_drone_v5.index, df_drone_v5['acceleration_drone_v5'], label='drone acceleration')
            # ax[2].plot(df_tracker.index, df_tracker['acceleration_tracker'], label='acceleration')
            # ax[2].set_ylabel('Acceleration [m/s${^2}$]')
            # ax[2].set_xlabel('Time [s]')
            # ax[2].grid()
            # ax[2].set_xlim([0,9])
            # plt.show()

        # _______________________________________________________________________________________________
        #bounding boxes generated by YOLO from wheelchair videos in normal condition and av5er application of the several filters
        # index|original|bright_1|bright_2|bright_3|dark_1|dark_2|dark_3|fog_1|fog_2|fog_3|rain_1|rain_2|rain_3
        #YOLO v5
        if (cam_w_yolo_v5== 'True') :
            df_wheelchair_temp_v5 = pd.read_csv(f'{folder_name}/bbox_wheelchair_v5.csv') #da cambiare nome su dataset
            
            
            if len(df_wheelchair_temp_v5.index)!= len(df_wheelchair_stamps.index):
                print("ERROR IN DATA DIMENSION FOR wheelchair VIDEO")
                print("exp_nr="+ str(exp_nr))
                print("filter="+ str(filter))
                #sys.exit("Error message")

            #Reset Dataframe index, correcting data type
            df_wheelchair_temp_v5 = df_wheelchair_temp_v5.reset_index(drop=True)
            

            df_wheelchair_v5 =pd.DataFrame()
            df_wheelchair_v5['timestamp'] =df_wheelchair_stamps

            #Filter selection between:original,bright_1,bright_2,bright_3,dark_1,dark_2,dark_3,fog_1,fog_2,fog_3,rain_1,rain_2,rain_3
            df_wheelchair_v5['bbox'] =df_wheelchair_temp_v5[filter]
            #distance computation
            df_wheelchair_v5['distance'] = ((0.24 * focal_v5) / df_wheelchair_v5['bbox'])                #time is converted from timestamps to seconds
            
            #time is converted from timestamps to seconds
            df_wheelchair_v5.index = (df_wheelchair_v5['timestamp'] - timestamp_track_min)/1_000_000_000

            df_wheelchair_v5.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True)

            #Speed and accelaration calculus for the tracker
            df_wheelchair_v5 = df_wheelchair_v5.loc[interval_start:interval_end]
            
            ###################df_wheelchair_v5 = df_wheelchair_v5.loc[:interval_end]
           
            
            # Delete the corrupted data from wheelchair camera outputs:
            df_wheelchair_clean = pd.DataFrame(columns=df_wheelchair_v5.columns).copy(deep=True)
            bad_index = []
            for i in np.arange(len(df_wheelchair_v5.index)):
                if abs(df_wheelchair_v5.index[i] - df_wheelchair_v5.index[i-1]) >= 0.01:
                    df_wheelchair_clean = pd.concat([df_wheelchair_clean, df_wheelchair_v5.iloc[i-1].to_frame().T])

                else:
                    index_mean = (df_wheelchair_v5.index[i] + df_wheelchair_v5.index[i - 1]) / 2
                    temp_dict ={}
                    temp_dict['index'] = index_mean
                    temp_dict['distance'] = (df_wheelchair_v5['distance'].iloc[i] + df_wheelchair_v5['distance'].iloc[i-1])/2
                    temp_dict['bbox'] = (df_wheelchair_v5['bbox'].iloc[i] + df_wheelchair_v5['bbox'].iloc[i-1])/2
                    temp_dict['absolute_timestamp'] = (df_wheelchair_v5['absolute_timestamp'].iloc[i] + df_wheelchair_v5['absolute_timestamp'].iloc[i-1])/2
                    bad_index.append(df_wheelchair_v5.index[i-1])
                    temp = pd.DataFrame(temp_dict, index=[temp_dict['index']]).copy(deep=True)
                    df_wheelchair_clean = pd.concat([df_wheelchair_clean, temp.drop(['index'], axis=1)])

            df_wheelchair_v5 = df_wheelchair_v5.drop(bad_index)
            ## Smoothing
            #Apply rolling average with the previous n values, 5 for the range sensor, and 2 for the cameras, in order to reduce outliers
            df_wheelchair_v5['distance'] = df_wheelchair_v5['distance'].rolling(window=2).mean()
            df_wheelchair_v5 = df_wheelchair_v5.rename(columns={'distance': 'distance_wheelchair','absolute_timestamp': 'absolute_timestamp_wheelchair', 'bbox': 'bbox_wheelchair'})



            #Speed and accelaration calculus
            df_wheelchair_v5['speed_wheelchair'] = np.insert(- np.diff(df_wheelchair_v5['distance_wheelchair'])/np.diff(df_wheelchair_v5.index), 0, 0)
            max_speed = 3.5
            min_speed = 0
            df_wheelchair_v5['speed_wheelchair'] = np.clip(df_wheelchair_v5['speed_wheelchair'], a_min=min_speed, a_max=max_speed)
            df_wheelchair_v5['acceleration_wheelchair'] = np.insert(np.diff(df_wheelchair_v5['speed_wheelchair'])/np.diff(df_wheelchair_v5.index), 0, 0)



            #danger function application
            df_wheelchair_v5['danger_wheelchair'] = df_wheelchair_v5.apply(lambda row: danger_function_composed(row, 'wheelchair'), axis=1)

            df_wheelchair_v5.rename(columns={'distance_wheelchair': 'distance_wheelchair_v5'}, inplace=True)
            df_wheelchair_v5.rename(columns={'speed_wheelchair': 'speed_wheelchair_v5'}, inplace=True)
            df_wheelchair_v5.rename(columns={'acceleration_wheelchair': 'acceleration_wheelchair_v5'}, inplace=True)
            df_wheelchair_v5.rename(columns={'danger_wheelchair': 'danger_wheelchair_v5'}, inplace=True)
            df_wheelchair_v5 = df_wheelchair_v5.drop(['bbox_wheelchair', 'absolute_timestamp_wheelchair'], axis=1)


            # fig, ax = plt.subplots(3,1)
            # # Index 4 Axes arrays in 4 subplots within 1 Figure: 
            # plt.title('wheelchair')
            # ax[0].plot(df_wheelchair_v5.index, df_wheelchair_v5['distance_wheelchair'], label='wheelchair distance') 
            # ax[0].plot(df_tracker.index, df_tracker['distance'], label='tracker distance') 
            # ax[0].set_ylabel('Distance [m]')
            # ax[0].set_xlabel('Time [s]')
            # ax[0].grid()
            # ax[0].set_xlim([0,9])

            # ax[1].plot(df_wheelchair_v5.index, df_wheelchair_v5['speed_wheelchair'], label='wheelchair speed')
            # ax[1].plot(df_tracker.index, df_tracker['speed_tracker'], label='speed') 
            # ax[1].set_ylabel('Speed [m/s]')
            # ax[1].set_xlabel('Time [s]')
            # ax[1].grid()
            # ax[1].set_xlim([0,9])

            # ax[2].plot(df_wheelchair_v5.index, df_wheelchair_v5['acceleration_wheelchair'], label='wheelchair acceleration')
            # ax[2].plot(df_tracker.index, df_tracker['acceleration_tracker'], label='acceleration')
            # ax[2].set_ylabel('Acceleration [m/s${^2}$]')
            # ax[2].set_xlabel('Time [s]')
            # ax[2].grid()
            # ax[2].set_xlim([0,9])
            # plt.show()

        # _______________________________________________________________________________________________
        #bounding boxes generated by YOLO from wheelchair videos in normal condition and aFTer application of the several filters
        # index|original|bright_1|bright_2|bright_3|dark_1|dark_2|dark_3|fog_1|fog_2|fog_3|rain_1|rain_2|rain_3
        #YOLO FT
        if (cam_w_yolo_FT== 'True') :
            df_wheelchair_temp_FT = pd.read_csv(f'{folder_name}/bbox_wheelchair_FT.csv') #da cambiare nome su dataset
            
            
            if len(df_wheelchair_temp_FT.index)!= len(df_wheelchair_stamps.index):
                print("ERROR IN DATA DIMENSION FOR wheelchair VIDEO")
                print("exp_nr="+ str(exp_nr))
                print("filter="+ str(filter))
                #sys.exit("Error message")

            #Reset Dataframe index, correcting data type
            df_wheelchair_temp_FT = df_wheelchair_temp_FT.reset_index(drop=True)
            

            df_wheelchair_FT =pd.DataFrame()
            df_wheelchair_FT['timestamp'] =df_wheelchair_stamps

            #Filter selection between:original,bright_1,bright_2,bright_3,dark_1,dark_2,dark_3,fog_1,fog_2,fog_3,rain_1,rain_2,rain_3
            df_wheelchair_FT['bbox'] =df_wheelchair_temp_FT[filter]
            #distance computation
            df_wheelchair_FT['distance'] = ((0.24 * focal_v8) / df_wheelchair_FT['bbox'])                #time is converted from timestamps to seconds
            
            #time is converted from timestamps to seconds
            df_wheelchair_FT.index = (df_wheelchair_FT['timestamp'] - timestamp_track_min)/1_000_000_000

            df_wheelchair_FT.rename(columns={'timestamp': 'absolute_timestamp'}, inplace=True)

            #Speed and accelaration calculus for the tracker
            df_wheelchair_FT = df_wheelchair_FT.loc[interval_start:interval_end]
            
            ##################df_wheelchair_FT = df_wheelchair_FT.loc[:interval_end]

            
            # Delete the corrupted data from wheelchair camera outputs:
            df_wheelchair_clean = pd.DataFrame(columns=df_wheelchair_FT.columns).copy(deep=True)
            bad_index = []
            for i in np.arange(len(df_wheelchair_FT.index)):
                if abs(df_wheelchair_FT.index[i] - df_wheelchair_FT.index[i-1]) >= 0.01:
                    df_wheelchair_clean = pd.concat([df_wheelchair_clean, df_wheelchair_FT.iloc[i-1].to_frame().T])

                else:
                    index_mean = (df_wheelchair_FT.index[i] + df_wheelchair_FT.index[i - 1]) / 2
                    temp_dict ={}
                    temp_dict['index'] = index_mean
                    temp_dict['distance'] = (df_wheelchair_FT['distance'].iloc[i] + df_wheelchair_FT['distance'].iloc[i-1])/2
                    temp_dict['bbox'] = (df_wheelchair_FT['bbox'].iloc[i] + df_wheelchair_FT['bbox'].iloc[i-1])/2
                    temp_dict['absolute_timestamp'] = (df_wheelchair_FT['absolute_timestamp'].iloc[i] + df_wheelchair_FT['absolute_timestamp'].iloc[i-1])/2
                    bad_index.append(df_wheelchair_FT.index[i-1])
                    temp = pd.DataFrame(temp_dict, index=[temp_dict['index']]).copy(deep=True)
                    df_wheelchair_clean = pd.concat([df_wheelchair_clean, temp.drop(['index'], axis=1)])

            df_wheelchair_FT = df_wheelchair_FT.drop(bad_index)
            ## Smoothing
            #Apply rolling average with the previous n values, 5 for the range sensor, and 2 for the cameras, in order to reduce outliers
            df_wheelchair_FT['distance'] = df_wheelchair_FT['distance'].rolling(window=2).mean()
            df_wheelchair_FT = df_wheelchair_FT.rename(columns={'distance': 'distance_wheelchair','absolute_timestamp': 'absolute_timestamp_wheelchair', 'bbox': 'bbox_wheelchair'})

            #Speed and accelaration calculus
            df_wheelchair_FT['speed_wheelchair'] = np.insert(- np.diff(df_wheelchair_FT['distance_wheelchair'])/np.diff(df_wheelchair_FT.index), 0, 0)
            max_speed = 3.5
            min_speed = 0
            df_wheelchair_FT['speed_wheelchair'] = np.clip(df_wheelchair_FT['speed_wheelchair'], a_min=min_speed, a_max=max_speed)
            df_wheelchair_FT['acceleration_wheelchair'] = np.insert(np.diff(df_wheelchair_FT['speed_wheelchair'])/np.diff(df_wheelchair_FT.index), 0, 0)

            #danger function application
            df_wheelchair_FT['danger_wheelchair'] = df_wheelchair_FT.apply(lambda row: danger_function_composed(row, 'wheelchair'), axis=1)


            df_wheelchair_FT.rename(columns={'distance_wheelchair': 'distance_wheelchair_FT'}, inplace=True)
            df_wheelchair_FT.rename(columns={'speed_wheelchair': 'speed_wheelchair_FT'}, inplace=True)
            df_wheelchair_FT.rename(columns={'acceleration_wheelchair': 'acceleration_wheelchair_FT'}, inplace=True)
            df_wheelchair_FT.rename(columns={'danger_wheelchair': 'danger_wheelchair_FT'}, inplace=True)
            df_wheelchair_FT = df_wheelchair_FT.drop(['bbox_wheelchair', 'absolute_timestamp_wheelchair'], axis=1)



            # fig, ax = plt.subplots(3,1)
            # # Index 4 Axes arrays in 4 subplots within 1 Figure: 
            # plt.title('wheelchair')
            # ax[0].plot(df_wheelchair_FT.index, df_wheelchair_FT['distance_wheelchair'], label='wheelchair distance') 
            # ax[0].plot(df_tracker.index, df_tracker['distance'], label='tracker distance') 
            # ax[0].set_ylabel('Distance [m]')
            # ax[0].set_xlabel('Time [s]')
            # ax[0].grid()
            # ax[0].set_xlim([0,9])

            # ax[1].plot(df_wheelchair_FT.index, df_wheelchair_FT['speed_wheelchair'], label='wheelchair speed')
            # ax[1].plot(df_tracker.index, df_tracker['speed_tracker'], label='speed') 
            # ax[1].set_ylabel('Speed [m/s]')
            # ax[1].set_xlabel('Time [s]')
            # ax[1].grid()
            # ax[1].set_xlim([0,9])

            # ax[2].plot(df_wheelchair_FT.index, df_wheelchair_FT['acceleration_wheelchair'], label='wheelchair acceleration')
            # ax[2].plot(df_tracker.index, df_tracker['acceleration_tracker'], label='acceleration')
            # ax[2].set_ylabel('Acceleration [m/s${^2}$]')
            # ax[2].set_xlabel('Time [s]')
            # ax[2].grid()
            # ax[2].set_xlim([0,9])
            # plt.show()

        # _______________________________________________________________________________________________
        ## Fusion Merge data
        #Data from all sensors is merged in a complete table via full outer merge on timestamp
        df_temp=df_tracker.copy(deep='True')
        if cam_w_yolo_FT=='True':
            df_temp = df_temp.merge(df_wheelchair_FT, on='timestamp', how='outer')
        if cam_w_yolo_v5=='True':
            df_temp = df_temp.merge(df_wheelchair_v5, on='timestamp', how='outer')
        if cam_d_yolo_FT=='True':
            df_temp = df_temp.merge(df_drone_FT, on='timestamp', how='outer')
        if cam_d_yolo_v5=='True':
            df_temp = df_temp.merge(df_drone_v5, on='timestamp', how='outer')
        if range_sens=='True':
            df_temp = df_temp.merge(df_range, on='timestamp', how='outer')
        
        df = df_temp.copy(deep='True')
        df.sort_values(by=['timestamp'])

        #print(df)

        df.index = np.round(df.index, 3)

        ## Resampling
        #To resample to 100 Hz index is truncated to the next hundreds of seconds, so rounded by excess. Then the timeline is completed with missing timestamps

        df_100 = df.copy()
        df_100.index= df_100.index*100
        df_100.index = np.trunc(df_100.index)
        df_100.index = df_100.index+1
        df_100.index = df_100.index/100
        
        interval_end_value = (np.trunc(interval_end*100)+1)/100
        interval_start_value = (np.trunc(interval_start*100)+1)/100


        #All timestamps which were not in the data are added to index
        to_add = np.arange(df_100.index.sort_values()[0], df_100.index.sort_values()[-1], step=0.01)
        to_add = np.round(to_add, 2)
        for item in to_add:
            if item not in df_100.index:
                new_row = pd.DataFrame(index=[item], columns=df_100.columns)
                new_row[:] = np.nan 
                df_100 = pd.concat([df_100, new_row])

        df_100.duplicated().sum()

        df_100['tracker_data'] = df_100['distance_tracker'].notnull()

        if cam_w_yolo_FT=='True':
            df_100['wheelchair_data_FT'] = df_100['distance_wheelchair_FT'].notnull()
        if cam_w_yolo_v5=='True':
            df_100['wheelchair_data_v5'] = df_100['distance_wheelchair_v5'].notnull()
        if cam_d_yolo_FT=='True':
           df_100['drone_data_FT'] = df_100['distance_drone_FT'].notnull()
        if cam_d_yolo_v5=='True':
           df_100['drone_data_v5'] = df_100['distance_drone_v5'].notnull()
        if range_sens=='True':
           df_100['range_data'] = df_100['distance_range'].notnull()
        
        df_100 = df_100.sort_index()

        #df_100_fusion=df_100_fusion.ffill(limit=5)



        

        #All data is forward filled, so that the last valid value is reported in following rows, until a new valid value is introduced
        
        #print('cam_w_yolo_FT firts value before fill=',df_100['distance_wheelchair_FT'])

        #df_100= df_100.ffill(axis=1)
        #print('cam_w_yolo_FT firts value before fill=',df_100)

        df_100= df_100.ffill(limit=4)
        df_100['distance_tracker']=df_100['distance_tracker'].ffill(limit=5)
        df_100[ 'speed_tracker']=df_100[ 'speed_tracker'].ffill(limit=5)
        df_100['acceleration_tracker']= df_100['acceleration_tracker'].ffill(limit=5)


        df_100['distance_tracker']=df_100['distance_tracker'].bfill(limit=5)
        df_100[ 'speed_tracker']=df_100[ 'speed_tracker'].bfill(limit=5)
        df_100['acceleration_tracker']= df_100['acceleration_tracker'].bfill(limit=5)
        df_100_fusion=df_100.copy(deep='True')
        #print('cam_w_yolo_FT firts value after fill=',df_100)

        max_value_distance=4
                #first value is set to 10 if the sensor doesn't see anything

        #print('cam_w_yolo_FT firts value before change=',df_100['distance_wheelchair_FT'])
        if cam_w_yolo_FT=='True':
            #if pd.isna(df_100['distance_wheelchair_FT'].iloc[0]):
            print('cam_w_yolo_FT firts value missing')
                # df_100.loc[df_100.index[0],'distance_wheelchair_FT']=max_value_tracker+1
                # df_100.loc[df_100.index[0],'speed_wheelchair_FT']=0
                # df_100.loc[df_100.index[0],'acceleration_wheelchair_FT']=0
                # print('cam_w_yolo_FT firts value=',df_100.loc[df_100.index[0],'distance_wheelchair_FT'])
            df_100['distance_wheelchair_FT']=df_100['distance_wheelchair_FT'].fillna(max_value_distance)
            df_100['speed_wheelchair_FT']=df_100['speed_wheelchair_FT'].fillna(0)
            df_100['acceleration_wheelchair_FT']=df_100['acceleration_wheelchair_FT'].fillna(0)
            df_100['danger_wheelchair_FT']=df_100['danger_wheelchair_FT'].fillna(0)

        if cam_w_yolo_v5=='True':
            #if pd.isna(df_100['distance_wheelchair_v5'].iloc[0]):
            print('cam_w_yolo_v5 firts value missing')
            df_100['distance_wheelchair_v5']=df_100['distance_wheelchair_v5'].fillna(max_value_distance)
            df_100['speed_wheelchair_v5']=df_100['speed_wheelchair_v5'].fillna(0)
            df_100['acceleration_wheelchair_v5']=df_100['acceleration_wheelchair_v5'].fillna(0)
            df_100['danger_wheelchair_v5']=df_100['danger_wheelchair_v5'].fillna(0)
   
        if cam_d_yolo_FT=='True':
            #if pd.isna(df_100['distance_drone_FT'].iloc[0]):
            print('cam_d_yolo_FT firts value missing')
            df_100['distance_drone_FT']=df_100['distance_drone_FT'].fillna(max_value_distance)
            df_100['speed_drone_FT']=df_100['speed_drone_FT'].fillna(0)
            df_100['acceleration_drone_FT']=df_100['acceleration_drone_FT'].fillna(0)
            df_100['danger_drone_FT']=df_100['danger_drone_FT'].fillna(0)

        if cam_d_yolo_v5=='True':
            #if pd.isna(df_100['distance_drone_v5'].iloc[0]):
            print('cam_d_yolo_v5 firts value missing')
            df_100['distance_drone_v5']=df_100['distance_drone_v5'].fillna(max_value_distance)
            df_100['speed_drone_v5']=df_100['speed_drone_v5'].fillna(0)
            df_100['acceleration_drone_v5']=df_100['acceleration_drone_v5'].fillna(0)
            df_100['danger_drone_v5']=df_100['danger_drone_v5'].fillna(0)

        if range_sens=='True': #DA RICONTROLLARE
           #if pd.isna(df_100['distance_range'].iloc[0]):
            df_100['range_sens']= df_100['range_sens'].ffill()
            if pd.isna(df_100['distance_range'].iloc[0]):
               df_100.loc[df_100.index[0],'distance_range']=max_value_tracker
               df_100.loc[df_100.index[0],'speed_range']=0
               df_100.loc[df_100.index[0],'acceleration_range']=0
               df_100['range_sens']= df_100['range_sens'].ffill()

        #print('cam_w_yolo_FT firts value after second fill=',df_100['distance_wheelchair_FT'])

        #print('cam_w_yolo_FT firts value after fill=',df_100['distance_wheelchair_FT'])

        #df_100 = df_100.loc[interval_start:interval_end]

        #################df_100 = df_100.loc[:interval_end_value]

        df_100 = df_100[~df_100.index.duplicated(keep='last')]
        df_100.index.duplicated().sum()
        

        # to save the proprocessed data in the file pre_fusion_data_{filter}.csv

        # Path  
        if os.path.isdir(f'{folder_name}/pre_fusion_data'):
            df_100.to_csv(f'{folder_name}/pre_fusion_data/pre_fusion_data_{filter}.csv')
        else:
            # Directory  
            directory = "pre_fusion_data"
            # Parent Directory path  
            parent_dir = f'{folder_name}'
            path = os.path.join(parent_dir, directory)  
            os.mkdir(path)
            df_100.to_csv(f'{folder_name}/pre_fusion_data/pre_fusion_data_{filter}.csv')



        #data preprocessed for fusion
        #df_100_fusion = df_100_fusion.loc[interval_start_value:interval_end_value]
        df_100_fusion = df_100_fusion[~df_100_fusion.index.duplicated(keep='last')]
        df_100_fusion.index.duplicated().sum()

        if os.path.isdir(f'{folder_name}/pre_fusion_data_forfusion'):
            df_100_fusion.to_csv(f'{folder_name}/pre_fusion_data_forfusion/pre_fusion_data_{filter}.csv')
        else:
            # Directory  
            directory = "pre_fusion_data_forfusion"
            # Parent Directory path  
            parent_dir = f'{folder_name}'
            path = os.path.join(parent_dir, directory)  
            os.mkdir(path)
            print(path)
            df_100_fusion.to_csv(f'{folder_name}/pre_fusion_data_forfusion/pre_fusion_data_{filter}.csv')
